In [ ]:
using Plots, LinearAlgebra, SparseArrays, BenchmarkTools, LaTeXStrings
using NAJ

In [ ]:
function tm(m1::Real, m2::Real, m3::Real, G::Real, r)
    local r12 = r[4:6] .- r[1:3]
    local r23 = r[7:9] .- r[4:6]
    local r31 = r[1:3] .- r[7:9] 
    
    local n12 = norm(r12)^3
    local n23 = norm(r23)^3
    local n31 = norm(r31)^3
    
    A = zeros(18, 18)
    
    for i in 1:9
        A[i, 9+i] = 1.0
    end

    II = [1.0 0 0; 0 1 0; 0 0 1]
    F12 = (G*m1*m2/n12) *II
    F23 = (G*m2*m3/n23) *II
    F31 = (G*m3*m1/n31) *II

    if m1 > 0.0
        A[10:12, 4:6] = F12/m1
        A[10:12, 1:3] += -F12/m1
        A[10:12, 7:9] = F31/m1
        A[10:12, 1:3] += -F31/m1
    end

    if m2 > 0.0
        A[13:15, 1:3] = F12/m2
        A[13:15, 4:6] += -F12/m2
        A[13:15, 7:9] = F23/m2
        A[13:15, 4:6] += -F23/m2
    end
    
    if m3 > 0.0
        A[16:18, 1:3] = F31/m3
        A[16:18, 7:9] += -F31/m3
        A[16:18, 4:6] = F23/m3
        A[16:18, 7:9] += -F23/m3
    end

    return A
end

function threebody(m1::Real, m2::Real, m3::Real, G::Real, r)
    A = tm(m1, m2, m3, G, r)
    return A*r
end

In [ ]:
norm([1;1;1]) - sqrt(3.)

In [ ]:
# m0 = 5.972e24
# v0 = 29.78e3
# r0 =147.10e9

# m1 = 1.0*m0
# m2 = 0.0 #1.0*m0
# m3 = 1.0e5*m0


# G =  6.67384e−11
# r1 = [1 ; 0; 0.0] .* r0
# r2 = [0 ; 4; 0.0] .* r0
# r3 = [0; 0; 0.0] .* r0
# v1 = [1; 0; 0.0] .* v0
# v2 = [1; 1; 0.0] .* v0
# v3 = [0; 0; 0.0] .* v0

m1 = 3.0
m2 = 4.0
m3 = 5.0

r0 = 1.0
v0 = 1.0
G = 1
r1 = [1; 3; 0] .* r0
r2 = [-2; -1; 0] .* r0
r3 = [1; -1; 0] .* r0
v1 = [0; 0; 0] .* v0
v2 = [0; 0; 0].* v0
v3 = [0; 0; 0] .* v0



s = [r1; r2; r3; v1 ;v2 ; v3]
tn, xn = ode_rk4((t, y)->threebody(m1, m2, m3, G, y), 0.0, s, 10000000, 1.0e-5)
# tn, xn = ode_euler((t, y)->threebody(m1, m2, m3, G, y), 0.0, s, 87000, 3600)

In [ ]:
# # a=Animation()
# anim = @animate for i in 1:100:1000000
#     pl = plot(xn[1,1:i], xn[2, 1:i], aspect_ratio = :equal, lw=3)
#     plot!(xn[4,1:i], xn[5, 1:i],   lw=3)
#     plot!(xn[7,1:i], xn[8, 1:i] ,  lw=3)
#     frame(a, pl)

# end
# gif(anim, "3body.gif", fps = 15)

In [ ]:
dd = 1:1000:1000000

p1 = plot(xn[1, dd ], xn[2, dd ], aspect_ratio = :equal, lw=3, color = :red, label = "m1", frame = :box, title = L"t \in [1, 10]", xlim=(-4, 4), ylim = (-4, 4))
plot!(xn[4, dd ], xn[5, dd ],   lw=3, color = :blue, label = "m2")
plot!(xn[7, dd ], xn[8, dd ] ,  lw=3, color = :green, label = "m3")

# dd = 1000000:100:1700000
# p2 = plot(xn[1, dd ], xn[2, dd ], aspect_ratio = :equal, lw=3, color = :red, label = "m1", frame = :box, title = L"t \in [10, 20]")
# plot!(xn[4, dd ], xn[5, dd ],   lw=3, color = :blue, label = "m2")
# plot!(xn[7, dd ], xn[8, dd ] ,  lw=3, color = :green, label = "m3")

# plot([p1 p2])

In [ ]:
dd = 1000000:100:2000000
pl = plot(xn[1, dd ], xn[2, dd ], aspect_ratio = :equal, lw=3, color = :red, label = "m1", frame = :box, title = L"t \in [10, 20]")
plot!(xn[4, dd ], xn[5, dd ],   lw=3, color = :blue, label = "m2")
plot!(xn[7, dd ], xn[8, dd ] ,  lw=3, color = :green, label = "m3")

In [ ]:
 T = (0.5 * m1) .* (xn[10,:].^2 + xn[11,:].^2 +xn[12, :] .^2) .+ (0.5 * m2) .*  (xn[13,:].^2 + xn[14,:].^2 +xn[15, :] .^2)  .+ (0.5 * m3) .*  (xn[16,:].^2 + xn[17,:].^2 +xn[18, :] .^2)
 r12 = sqrt.((xn[1,:] .- xn[4, :]).^2 + (xn[2,:] .- xn[5,:]).^2 + (xn[3, :] .- xn[6,:]).^2)
 r23 = sqrt.((xn[7,:] .- xn[4, :]).^2 + (xn[8,:] .- xn[5,:]).^2 + (xn[8, :] .- xn[6,:]).^2)
 r31 = sqrt.((xn[1,:] .- xn[7, :]).^2 + (xn[2,:] .- xn[8,:]).^2 + (xn[3, :] .- xn[9,:]).^2)
 V = -1.0 .* ((m1*m2)./ r12 .+ (m2*m3) ./ r23 .+  (m1*m3)./r31)

In [ ]:
plot( T[1:100:end], label = L"K", ylim = (-10, 100), xlim = (1, 25000))
# plot!( V[1:100:end], label = L"V")
# plot!( (T+V)[1:100:end], label = L"K+V")
plot!(r12[1:100:end] .* 10, label = L"r_{12} \times 10")
plot!(r23[1:100:end] .* 10, label = L"r_{23} \times 10")
plot!(r31[1:100:end] .* 10, label = L"r_{31} \times 10")


In [ ]:
plot(T[100:100:end], label = L"K", yaxis=:log)
plot!(abs.(V[100:100:end]), label = L"V")

In [ ]:
norm([1; 2; 3])